In [1]:
from jsdconfig import JSDConfig
from jsdcontroller import JSDController
from jsdmodel import JSDTableModel

config = JSDConfig()
data_source_list = config.data['data sources']
# w = JsdWindow(data_source_list)  # Note: We should have the controller populate this once the tablemodel is loaded
jsd_controller = JSDController(None,
                               JSDTableModel(data_source_list, config.data.get('custom age ranges', None)),
                               config)

AttributeError: 'NoneType' object has no attribute 'add_data_source'

In [2]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from ipywidgets import VBox, HBox, Label
from jsdcontroller import JSDController  # Assuming the backend JSDController is accessible
from PySide6.QtCore import Signal, QObject

# Define the data selection group box
class DataSelectionGroupBox(QObject):
    category_changed = Signal(str)
    file_selection_changed = Signal()

    def __init__(self, jsd_controller):
        super().__init__()
        self.jsd_controller = jsd_controller
        self.file_comboboxes = []
        self.category_combobox = widgets.Dropdown(
            options=[],
            value=None,
            description='Attribute:',
            disabled=True,
        )
        self.layout = VBox()
        self._setup_layout()
        self._initialize_data_sources()

        # Connect signals
        self.category_combobox.observe(self.on_category_changed, names='value')
        for combobox in self.file_comboboxes:
            combobox.observe(self.on_file_selection_changed, names='value')

    def _setup_layout(self):
        self.layout.children = [self.category_combobox]

    def _initialize_data_sources(self):
        # Initialize file comboboxes based on data sources from JSDController
        for index, data_source in enumerate(self.jsd_controller.jsd_model.data_sources.values()):
            combobox = widgets.Dropdown(
                options=list(data_source.sheets.keys()),
                value=list(data_source.sheets.keys())[0],
                description=f'Data Source {index + 1}:',
                disabled=False
            )
            self.file_comboboxes.append(combobox)
            self.layout.children += (combobox,)
            combobox.observe(self.on_file_selection_changed, names='value')
        
        # Initialize category combobox based on the current data selection
        if self.file_comboboxes:
            self.update_category_combobox()

    def update_category_combobox(self):
        # Update category combobox based on selected data sources
        selected_sheets = [combobox.value for combobox in self.file_comboboxes]
        common_categories = set.intersection(*[set(self.jsd_controller.jsd_model.data_sources[combobox.description].sheets.keys()) for combobox in self.file_comboboxes])
        self.category_combobox.options = list(common_categories)
        self.category_combobox.disabled = False

    def on_category_changed(self, change):
        self.category_changed.emit(change['new'])

    def on_file_selection_changed(self, change):
        self.file_selection_changed.emit()
        self.update_category_combobox()

    def display(self):
        return self.layout

# Create instance of DataSelectionGroupBox
data_selection_groupbox = DataSelectionGroupBox(jsd_controller)

# Define a function to plot the timeline chart
def plot_timeline_chart():
    plt.figure(figsize=(10, 6))
    category = data_selection_groupbox.category_combobox.value
    for combobox in data_selection_groupbox.file_comboboxes:
        if combobox.value:
            file_data = jsd_controller.jsd_model.data_sources[combobox.description].sheets[category].df
            sns.lineplot(data=file_data, x=file_data.index, y=category, label=f'{combobox.description}: {category}')
    plt.xlabel('Index')
    plt.ylabel('Values')
    plt.title('JSD Timeline Chart')
    plt.grid(True)
    plt.legend()
    plt.show()

# Define button to trigger the timeline chart plotting
plot_button = widgets.Button(description='Plot Timeline Chart')
output_timeline = widgets.Output()

def on_plot_button_clicked(b):
    with output_timeline:
        output_timeline.clear_output(wait=True)
        plot_timeline_chart()

plot_button.on_click(on_plot_button_clicked)

# Connect the category change signal to automatically update the chart
data_selection_groupbox.category_changed.connect(on_plot_button_clicked)

# Display the widgets
display(data_selection_groupbox.display(), plot_button, output_timeline)


ValueError: jsd_view must be a valid JsdWindow instance